In [1]:
import matplotlib.pyplot as plt 
from PIL import ImageDraw , Image , ImageFont
import numpy as np
import pandas as pd
import copy
import math
import operator
from random import seed
from random import random

In [2]:
#############################################
#Initially used for the HW 10 report, finds how many people used a certain line on Metallica day and then not on a Metallica day
dataBus = pd.read_csv("bus_validation_data/2019_july/11.07_20.07.csv")

dataBusApplied = dataBus.apply(lambda x: True if x['Liin'] == "7" and x["Aeg"].split(" ")[0]=="2019-07-11" else False , axis=1)
RowsBasic = len(dataBusApplied[dataBusApplied == True].index)
print(RowsBasic)

dataBusApplied = dataBus.apply(lambda x: True if x['Liin'] == "7" and x["Aeg"].split(" ")[0]=="2019-07-18" else False , axis=1)
rowsMetallica = len(dataBusApplied[dataBusApplied == True].index)
print(rowsMetallica)
#############################################

1044
1288


In [3]:
#START OF IMPORTING THE DATA/CLEANING THE DATA
#Users of the notebook should either delete this part completely or get the actual data and then making sure that
    #the files are in the right place(directory wise) and with the correct name
#import all the bike stuff in here
dataBikeLocJuly1 = pd.read_csv("bicycle_data/locations_201907_part1.csv")
dataBikeLocJuly2 = pd.read_csv("bicycle_data/locations_201907_part2.csv")
dataBikeLocAugust1 = pd.read_csv("bicycle_data/locations_201908_part1.csv")
dataBikeLocAugust2 = pd.read_csv("bicycle_data/locations_201908_part2.csv")
dataBikeLocSeptember1 = pd.read_csv("bicycle_data/locations_201909.csv")
dataBikeRouteJuly = pd.read_csv("bicycle_data/routes_201907.csv")
dataBikeRouteAugust = pd.read_csv("bicycle_data/routes_201908.csv")
dataBikeRouteSeptember = pd.read_csv("bicycle_data/routes_201909.csv")
dataBikeRoute18Jul =  pd.read_csv("Smart Bike Tartu_july 2019/routes_20190718.csv")
dataBikeLoc18Jul = pd.read_csv("Smart Bike Tartu_july 2019/locations_20190718.csv")
dataBikeRoute19Jul =  pd.read_csv("Smart Bike Tartu_july 2019/routes_20190719.csv")
dataBikeLoc19Jul = pd.read_csv("Smart Bike Tartu_july 2019/locations_20190719.csv")
dataBikeRoute25Jul =  pd.read_csv("Smart Bike Tartu_july 2019/routes_20190725.csv")
dataBikeLoc25Jul = pd.read_csv("Smart Bike Tartu_july 2019/locations_20190725.csv")
dataBikeRoute26Jul =  pd.read_csv("Smart Bike Tartu_july 2019/routes_20190726.csv")
dataBikeLoc26Jul = pd.read_csv("Smart Bike Tartu_july 2019/locations_20190726.csv")

#Put all that bike stuff into one single list
listOfEverything = [dataBikeLocJuly1,dataBikeLocJuly2,dataBikeLocAugust1,dataBikeLocAugust2,dataBikeLocSeptember1,dataBikeRouteJuly,
                   dataBikeRouteAugust,dataBikeRouteSeptember,dataBikeLoc18Jul,dataBikeRoute18Jul,dataBikeRoute19Jul,dataBikeLoc19Jul,
                   dataBikeRoute25Jul,dataBikeLoc25Jul,dataBikeRoute26Jul,dataBikeLoc26Jul]

In [4]:
#1. drop the undefined rows, as there are like a handful (minscule in comparison to the overall size)
#2. For some forsaken reason there are some weird +00 things at the end of MOST(not all for some other reason) time cells in location data
        #So we are removing that, because it helps when we try to compare things later (probably)
    #Also helps with the classic IT OCD
for i in range(len(listOfEverything)): #Drops missing data rows (as there were only a few, I thought we can just remove them)
    listOfEverything[i] = listOfEverything[i].dropna() #So use this list when wanting to do something with the data
    if i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 8 or i == 11 or i == 13 or i == 15: #where i = location dataset
        for j in range(listOfEverything[i].shape[0]):
            listOfEverything[i].iat[j,5] = listOfEverything[i].iat[j,5].split("+")[0]










In [5]:
#Create different dataframes for a whole set / just for Metallica
allTheLocRows = pd.concat([listOfEverything[0],listOfEverything[1],listOfEverything[2],listOfEverything[3],listOfEverything[4],listOfEverything[8],listOfEverything[11],listOfEverything[13],listOfEverything[15]])
allTheRouteRows = pd.concat([listOfEverything[5],listOfEverything[6],listOfEverything[7],listOfEverything[9],listOfEverything[10],listOfEverything[12],listOfEverything[14]])
justMetallicaRoute = listOfEverything[9]
justMetallicaLoc = listOfEverything[8]

#Helps with printing
pd.set_option('display.max_rows', 5000)

#Dropping unneeded columns and duplicates (there are some)
allTheLocRows = allTheLocRows.drop("cyclenumber",axis=1)
allTheLocRows = allTheLocRows.drop_duplicates()
justMetallicaLoc = justMetallicaLoc.drop("cyclenumber",axis=1)

allTheRouteRows = allTheRouteRows.drop(["cyclenumber","rfidnumber","CycleType","costs"], axis=1)
allTheRouteRows = allTheRouteRows.drop_duplicates()
justMetallicaRoute = justMetallicaRoute.drop(["cyclenumber","rfidnumber","CycleType","costs"], axis=1)

#Sorting the values to be in order
allTheLocRows = allTheLocRows.sort_values(by=["route_code","coord_date","coord_time"])
justMetallicaLoc = justMetallicaLoc.sort_values(by=["route_code","coord_date","coord_time"])

allTheRouteRows = allTheRouteRows.sort_values(by=["route_code","unlockedat","unlockedattime"])
justMetallicaRoute = justMetallicaRoute.sort_values(by=["route_code","unlockedat","unlockedattime"])

#As concating makes a joke out of indexes, I will reindex the data, as that way loc will work kind of well
allTheLocRows.index = range(0,allTheLocRows.shape[0])
justMetallicaLoc.index = range(0,justMetallicaLoc.shape[0])
allTheRouteRows.index = range(0,allTheRouteRows.shape[0])
justMetallicaRoute.index = range(0,justMetallicaRoute.shape[0])


In [6]:
#Trying to delete days where there is lacking data (less than 50 different rides OR less that 6 hours of the day covered)
def dayDeleteFinder(dataframe): #Returns a list of deletable days
    dailyrides = {}
    dailyhours = {}
    lasthour = -55
    lastrideid = -55
    for i in range(dataframe.shape[0]):
        if dataframe.iat[i,0] != lastrideid:
            key = dataframe.iat[i,3]
            if key in dailyrides:
                dailyrides[key] += 1
            else:
                dailyrides[key] = 1
            lastrideid = dataframe.iat[i,0]
            if key in dailyhours:
                dailyhours[key].add(dataframe.iat[i,4].split(":")[0])
            else:
                dailyhours[key] = set()
            lasthour = dataframe.iat[i,4].split(":")[0]
        elif dataframe.iat[i,4].split(":")[0] != lasthour:
            key = dataframe.iat[i,3]
            if key in dailyhours:
                dailyhours[key].add(dataframe.iat[i,4].split(":")[0])
            else:
                dailyhours[key] = set()
    returnable = []
    for key in dailyrides:
        if dailyrides[key] < 50 or len(dailyhours[key]) < 6:
            returnable.append(key)
    return returnable

#Apply the row removal to created systems
deletableDaysMetallica = dayDeleteFinder(justMetallicaLoc)
justMetallicaLoc = justMetallicaLoc[~justMetallicaLoc.coord_date.isin(deletableDaysMetallica)]
justMetallicaRoute = justMetallicaRoute[~justMetallicaRoute.unlockedat.isin(deletableDaysMetallica)]

deletableDaysEverything = dayDeleteFinder(allTheLocRows)
allTheLocRows = allTheLocRows[~allTheLocRows.coord_date.isin(deletableDaysEverything)]
allTheRouteRows = allTheRouteRows[~allTheRouteRows.unlockedat.isin(deletableDaysEverything)]

In [7]:
#Creating csv-files of cleaned data (for team-distribution purposes)
justMetallicaLoc.to_csv("justMetallica1807BikeLocationData.csv")
justMetallicaRoute.to_csv("justMetallica1807BikeRouteData.csv")
allTheRouteRows.to_csv("allTheBikeRouteData.csv")
allTheLocRows.to_csv("allTheLocRouteData.csv")

In [8]:
#Reading in the given bicycle station data and altering some rows (for convenience sake we replace certain data in 2 rows to make the data more standarized)
    #This helps us later on when it comes to drawing the stations on the map
StopCoordinates = pd.read_csv("SPOILER_bicycle_stations.csv")
StopCoordinates.iloc[0,1] = "GeoCircle"
StopCoordinates.iloc[0,2] = [58.38079076345506, 26.724423279705707, 0.00031971597835600996]
StopCoordinates.iloc[68,1] = "GeoCircle"
StopCoordinates.iloc[68,2] = [58.39785674147324, 26.740054525855093, 0.00031971597835600996]

In [9]:
#Setting up "global" variables
widthHeightFinder = Image.open("MapOfTartuModified.png")
width, height = widthHeightFinder.size

In [10]:
#Resets a 2D list elements to be 0-s
def resetTheCoordinatesFields(ListOfPixels):
    for i in range(len(ListOfPixels)):
        for j in range(len(ListOfPixels[i])):
            ListOfPixels[i][j] = 0
        
def confirmThatEveryPixelIsZero(pixelList): #Was and can be used at times for checks (as it seems like some days are way too similar)
    for i in range(len(pixelList)):
        for j in range(len(pixelList[i])):
            if pixelList[i][j] != 0:
                return False
    return True

In [11]:
#Calculating pixel coordinates on the map from real-world latitude/longitude
def calculatePixelXFromCoordinateX(X): #LIDL math to lineary convert the coordinate data to pixel data(interval [0,(width or length) -1])
    left = 26.658194 #The four direction numbers indicate the map edges and the real life coordinate data for the map edges (found manually)
    right = 26.781312
    return round((X - left)*(2869/(right - left)))
def calculatePixelYFromCoordinateY(Y): #Same thing for the Y (latitude)
    bottom = 58.342406
    top = 58.414036
    return abs(3185 - round((Y - bottom)*(3185/(top - bottom))))

In [12]:
#Dataframe altering/searching
def createLocDataframeBasedOnDate(date): #It creates a Dataframe of bike locations based on given date
    return allTheLocRows.loc[(allTheLocRows['coord_date'] == date)]

In [13]:
#Drawing the bike stops on the map
def calculateGeoCircleRadius(radius): #Finds what the bike station radius should be on the drawable map in pixels based on given radius in coordinates(assumably)
    left = 26.658194
    right = 26.781312
    bottom = 58.342406
    top = 58.414036
    #No idea in what units the radius is counted in, so I am using some pseudo-coordinate calculations, assuming that the radius is counted in coordinate length
    divider1 = (right-left) / 2869
    divider2 = (top-bottom) / 3185
    actualDivider = (divider1 + divider2) / 2
    return round(radius/actualDivider)

def drawStationsOnTheMap(image): #Draws (does not save) the bike stations on the map (in this case parameter image) for the classical every route in a day dataframe
    picture = image
    draw = ImageDraw.Draw(picture)
    colour = (0,0,0,255) #Currently black
    for k, row in StopCoordinates.iterrows():
        if(k == 0 or k == 68): #The rows that we manually corrected
            coordinateInformation = row["AreaPoints"]
        else:
            coordinateInformationString = row["AreaPoints"]
            coordinateInformationString = coordinateInformationString.replace("[","")
            coordinateInformationString = coordinateInformationString.replace("]","")
            coordinateInformation = coordinateInformationString.split(",")
            
        latitude = calculatePixelYFromCoordinateY(float(coordinateInformation[0]))
        longitude = calculatePixelXFromCoordinateX(float(coordinateInformation[1]))
        radius = 1.2 * calculateGeoCircleRadius(float((coordinateInformation[2])))
        font = ImageFont.truetype("comicbd.ttf", round(radius*1.45))

        draw.ellipse((longitude-radius, latitude-radius, longitude+radius, latitude+radius), fill=colour)
        
        clr=(128,128,128) #Text outline (gray at the moment)
        txt=str(k)
        if(k >= 10):
            txtX=longitude-0.8*radius
            txtY=latitude-0.95*radius
            
            draw.text((txtX-2, txtY-2), txt,clr,font=font) #OUTLINE
            draw.text((txtX+2, txtY-2), txt,clr,font=font) #OUTLINE
            draw.text((txtX+2, txtY+2), txt,clr,font=font) #OUTLINE
            draw.text((txtX-2, txtY+2), txt,clr,font=font) #OUTLINE
            draw.text((txtX, txtY),txt,(255,255,255),font=font) # MAIN
        else:
            txtX=longitude-0.3*radius
            txtY=latitude-0.95*radius
            
            draw.text((txtX-2, txtY-2), txt,clr,font=font)
            draw.text((txtX+2, txtY-2), txt,clr,font=font)
            draw.text((txtX+2, txtY+2), txt,clr,font=font)
            draw.text((txtX-2, txtY+2), txt,clr,font=font)
            draw.text((txtX, txtY),txt,(255,255,255),font=font)
    return picture

def drawStationsOnMapColoured(image,dictOfStops,filename): #This draws and saves the stations on the map, part of the top 12 routes finding method
    #The dictOfStops is a dictionary with keys being bike stop names and values being tuples of RGB colours.
    picture = image
    draw = ImageDraw.Draw(picture)
    rowsThatAreNeeded = StopCoordinates.loc[StopCoordinates["name"].isin(dictOfStops)]
    for k, row in rowsThatAreNeeded.iterrows():
        if(row["name"] == "Magistri" or row["name"] == "Metallica parkla"):
            coordinateInformation = row["AreaPoints"]
        else:
            coordinateInformationString = row["AreaPoints"]
            coordinateInformationString = coordinateInformationString.replace("[","")
            coordinateInformationString = coordinateInformationString.replace("]","")
            coordinateInformation = coordinateInformationString.split(",")
            
        latitude = calculatePixelYFromCoordinateY(float(coordinateInformation[0]))
        longitude = calculatePixelXFromCoordinateX(float(coordinateInformation[1]))
        radius = 1.2 * calculateGeoCircleRadius(float((coordinateInformation[2])))
        font = ImageFont.truetype("comicbd.ttf", round(radius*1.45))
        colour = dictOfStops[row["name"]]
        
        draw.ellipse((longitude-radius-2, latitude-radius-2, longitude+radius+2, latitude+radius+2), fill=(0,0,0)) #Black outer edge
        draw.ellipse((longitude-radius, latitude-radius, longitude+radius, latitude+radius), fill=colour) #Inner colour
        
        clr=(0,0,0) #Text outline
        txt=str(k)
        if(k >= 10):
            txtX=longitude-0.8*radius
            txtY=latitude-0.95*radius
            
            draw.text((txtX-2, txtY-2), txt,clr,font=font) #Draws a mediocre outline around the text
            draw.text((txtX+2, txtY-2), txt,clr,font=font)
            draw.text((txtX+2, txtY+2), txt,clr,font=font)
            draw.text((txtX-2, txtY+2), txt,clr,font=font)
            draw.text((txtX, txtY),txt,(255,255,255),font=font)
        else:
            txtX=longitude-0.3*radius
            txtY=latitude-0.95*radius
            
            draw.text((txtX-2, txtY-2), txt,clr,font=font)
            draw.text((txtX+2, txtY-2), txt,clr,font=font)
            draw.text((txtX+2, txtY+2), txt,clr,font=font)
            draw.text((txtX-2, txtY+2), txt,clr,font=font)
            draw.text((txtX, txtY),txt,(255,255,255),font=font)
    picture.save(filename)
    picture.show()

In [14]:
#Picture displaying, editing and saving
def displayAndSavePicture(ListOfPixels,filename): #Displays the picture with added colours from the list of pixels
    picture = Image.open("MapOfTartuModified.png")
    width, height = picture.size
    pixels = picture.load()
    for x in range(width):
        for y in range(height):
            current_color = pixels[x,y]
            current_color = list(current_color)
            current_color[0] = min(current_color[0] + ListOfPixels[y][x], 255)
            current_color[1] = max(current_color[1] - ListOfPixels[y][x], 0)
            current_color[2] = max(current_color[2] - ListOfPixels[y][x], 0)
            new_color = tuple(current_color)
            picture.putpixel( (x,y), new_color)
    
    picture = drawStationsOnTheMap(picture)
    picture.save(filename)
    picture.show()
    
def editPictureColor(ListOfPixels,image,color): #Sometimes we want to edit a picture bit by bit, so this method is for those situations
    picture = image
    width, height = picture.size
    pixels = picture.load()
    for x in range(width):
        for y in range(height):
            if ListOfPixels[y][x] > 0:
                picture.putpixel( (x,y), color)

In [15]:
def lineCoefficientFinder(X, Y): #Used for finding the most linear path between two points on the map in the form of a function y=ax+b
    return np.polyfit(X, Y, 1)

def lineDrawerOnTheMap(Point1,Point2,listOfPixels): #Used for marking the path between two points on the map (within the list of pixels)
    if(min(Point1[0],Point2[0]) < 0 or max(Point1[0],Point2[0]) >=2870 or min(Point1[1],Point2[1])<0 or max(Point1[1],Point2[1]) >= 3186):
        return
    if(Point1[0] == Point2[0]): #If the only change is up/down
        if(Point1[1] != Point2[1]): #If there was some change
            GlobalDrawnPoints = {}
            for i in range(int(min(Point1[1],Point2[1])),int(max(Point1[1],Point2[1]) + 1)):
                gradientLineDrawer(int(Point1[0]),int(i),GlobalDrawnPoints,listOfPixels)
    else:
        coefficients = lineCoefficientFinder([Point1[0],Point2[0]],[Point1[1],Point2[1]])
        GlobalDrawnPoints = {}
        lastY = -5
        lastX = -5
        for i in np.arange(min(Point1[0],Point2[0]),max(Point1[0],Point2[0]) + 0.1,0.1):
            currentY = int(round(i*coefficients[0] + coefficients[1]))
            currentX  = int(math.floor(i))
            if currentY != lastY or currentX != lastX:
                gradientLineDrawer(currentX,currentY,GlobalDrawnPoints,listOfPixels)
                lastY = currentY
                lastX = currentX
                
def lineDrawerOnTheMapTop(Point1,Point2,listOfPixels): #Same thing as above, but used when it comes to the top 12 routes dataframe
    if(min(Point1[0],Point2[0]) < 0 or max(Point1[0],Point2[0]) >=2870 or min(Point1[1],Point2[1])<0 or max(Point1[1],Point2[1]) >= 3186):
        return
    if(Point1[0] == Point2[0]):
        if(Point1[1] != Point2[1]):
            GlobalDrawnPoints = {}
            for i in range(int(min(Point1[1],Point2[1])),int(max(Point1[1],Point2[1]) + 1)):
                gradientLineDrawer(int(Point1[0]),int(i),GlobalDrawnPoints,listOfPixels)
    else:
        coefficients = lineCoefficientFinder([Point1[0],Point2[0]],[Point1[1],Point2[1]])
        GlobalDrawnPoints = {}
        lastY = -5
        lastX = -5
        for i in np.arange(min(Point1[0],Point2[0]),max(Point1[0],Point2[0]) + 0.05,0.05):
            currentY = int(round(i*coefficients[0] + coefficients[1]))
            currentX  = int(math.floor(i))
            if currentY != lastY or currentX != lastX:
                listOfPixels[currentY][currentX] += 1
                lastY = currentY
                lastX = currentX

def gradientLineDrawer(X,Y,DrawnPoints,listOfPixels): #Instead of drawing a one pixel constant strength line, this one draws something more akin to a brush type line
    for i in range(-2,3):
        
        if(X+i >=2870 or X+i < 0):
            continue
        
        for j in range(-2+abs(i),abs(-2+abs(i))+1):
            
            if(Y+j >=3186 or Y+j < 0):
                continue
            
            if abs(i)+abs(j) == 0:
                addableValue = 4
            elif abs(i) + abs(j) == 1:
                addableValue = 2
            else:
                addableValue = 1
                
                
            if (Y+j,X+i) not in DrawnPoints:
                listOfPixels[Y+j][X+i] += addableValue
                DrawnPoints[(Y+j,X+i)] = addableValue
            elif DrawnPoints[(Y+j,X+i)] < addableValue:
                listOfPixels[Y+j][X+i] += addableValue - DrawnPoints[Y+j,X+i]
                DrawnPoints[(Y+j,X+i)] = addableValue

In [16]:
#Ways to normalise the ListOfPixels data in a linear way (to [0,255])
def normalizePixelList(ListOfPixels):
    ListOfPixelsCopy = copy.deepcopy(ListOfPixels)
    maxfind = 0
    for i in range(len(ListOfPixelsCopy)):
         for j in range(len(ListOfPixelsCopy[i])):
            if ListOfPixelsCopy[i][j] > maxfind:
                maxfind = ListOfPixelsCopy[i][j]
    if maxfind == 0:
        return ListOfPixelsCopy
    coefficient = 255 / maxfind
    for i in range(len(ListOfPixelsCopy)):
            for j in range(len(ListOfPixelsCopy[i])):
                ListOfPixelsCopy[i][j] = round(ListOfPixelsCopy[i][j] * coefficient)
    return ListOfPixelsCopy

def normalizePixelListNotLinear(ListOfPixels): #Currently we mostly use this one, as it shows most roads but also highlights the most used ones
    #This method also normalizes the data, but in an exponential way. So most routes have the same colour, but very very used roads are a lot redder
                #and very rarely used roads are invisible/bearly visible
    ListOfPixelsCopy = copy.deepcopy(ListOfPixels)
    total = 0
    nonZeroFields = 0
    maxfind = 0
    for i in range(len(ListOfPixelsCopy)):
         for j in range(len(ListOfPixelsCopy[i])):
            total += ListOfPixelsCopy[i][j]
            if ListOfPixelsCopy[i][j] > 0:
                nonZeroFields += 1
            if ListOfPixelsCopy[i][j] > maxfind:
                maxfind = ListOfPixelsCopy[i][j]
    if nonZeroFields == 0:
        return ListOfPixelsCopy
    average = total / nonZeroFields
    normalRoadRed = 30
    upperCoefficient = (255-normalRoadRed)/((maxfind-average)**2) * 3
    lowerCoefficient = -3*(normalRoadRed/(average**2))
    for i in range(len(ListOfPixelsCopy)):
         for j in range(len(ListOfPixelsCopy[i])):
                if ListOfPixelsCopy[i][j] >= average:
                    ListOfPixelsCopy[i][j] = round(upperCoefficient*((ListOfPixelsCopy[i][j]-average)**2) + normalRoadRed )
                else:
                    if(ListOfPixelsCopy[i][j] == 0):
                        continue
                    ListOfPixelsCopy[i][j] = max(round(lowerCoefficient*((average-ListOfPixelsCopy[i][j])**2) + normalRoadRed ),0)
    return ListOfPixelsCopy

In [17]:
#Methods that control the whole process of drawing routes on the map

#MAIN METHOD
#Given an dataframe of bike location data and a fileName, this method maps the total bike usage
#The idea behind it is that the redder the part of the map, the more bikes rode through it.
#It shows and saves 3 maps in total, where the data has been normalized in different ways:
    #These include not normalizing, linear normalization, exponential normalization
def EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalization(dataframe,fileName): #Overall (also saves and displayed the image)
    routeC = 999
    coordinates = [999,999]
    pixelList = [[0 for _ in range(width)] for _ in range(height)]
    fileNameNoEXP = fileName+"noEXP.png"
    fileNameEXP = fileName+"withEXP.png"
    fileNameLinear = fileName+"linear.png"
    resetTheCoordinatesFields(pixelList)
    for i in range(dataframe.shape[0]):
        row = dataframe.iloc[i]
        if row[0] != routeC:
            routeC = row[0]
            coordinates = [row[1],row[2]]
        else:
            oldPoint = [calculatePixelXFromCoordinateX(coordinates[1]),calculatePixelYFromCoordinateY(coordinates[0])]
            newPoint = [calculatePixelXFromCoordinateX(row[2]),calculatePixelYFromCoordinateY(row[1])]
            if max(abs(oldPoint[0]-newPoint[0]),abs(oldPoint[1]-newPoint[1])) > 1: #This removes the problem of bikes just sitting around doing nothing
                lineDrawerOnTheMap(oldPoint,newPoint,pixelList)             
                coordinates = [row[1],row[2]]
    displayAndSavePicture(pixelList,fileNameNoEXP)
    displayAndSavePicture(normalizePixelListNotLinear(pixelList),fileNameEXP)
    displayAndSavePicture(normalizePixelList(pixelList),fileNameLinear)


#MAIN METHOD
#The idea is the same as before, except this one is made for dataframes that are made of many days and we want to make an average map of those days
def EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalizationAverage(dataframe,amountOfDays,fileName): #Overall (also saves and displayed the image)
    routeC = 999
    coordinates = [999,999]
    pixelList = [[0 for _ in range(width)] for _ in range(height)]
    fileNameNoEXP = fileName+"noEXP.png"
    fileNameEXP = fileName+"withEXP.png"
    fileNameLinear = fileName+"linear.png"
    resetTheCoordinatesFields(pixelList)
    for i in range(dataframe.shape[0]):
        row = dataframe.iloc[i]
        if row[0] != routeC:
            routeC = row[0]
            coordinates = [row[1],row[2]]
        else:
            oldPoint = [calculatePixelXFromCoordinateX(coordinates[1]),calculatePixelYFromCoordinateY(coordinates[0])]
            newPoint = [calculatePixelXFromCoordinateX(row[2]),calculatePixelYFromCoordinateY(row[1])]
            if max(abs(oldPoint[0]-newPoint[0]),abs(oldPoint[1]-newPoint[1])) > 1: #This removes the problem of bikes just sitting around doing nothing
                lineDrawerOnTheMap(oldPoint,newPoint,pixelList)             
                coordinates = [row[1],row[2]]

    for i in range(len(pixelList)):
        for j in range(len(pixelList[i])):
            pixelList[i][j] = round(pixelList[i][j]/amountOfDays)
    displayAndSavePicture(pixelList,fileNameNoEXP)
    displayAndSavePicture(normalizePixelListNotLinear(pixelList),fileNameEXP)
    displayAndSavePicture(normalizePixelList(pixelList),fileNameLinear)
            
    
def completeCoordinatesFieldsColor(dataframe,pixelList,picture,color):
    #An assisting method for TOP 12 and N longest runs methods, does not display/save the image)
    #It changes the pixel data having dataframe (pixelList) to have information for the currect TOP{1,2,3,....12} / {0,1,2,...,N} road
    #Then it edits the given picture to have those routes on them (with the given colour)
    routeC = 999
    coordinates = [999,999]
    resetTheCoordinatesFields(pixelList)
    for i in range(dataframe.shape[0]):
        row = dataframe.iloc[i]
        if row[0] != routeC:
            routeC = row[0]
            coordinates = [row[1],row[2]]
        else:
            oldPoint = [calculatePixelXFromCoordinateX(coordinates[1]),calculatePixelYFromCoordinateY(coordinates[0])]
            newPoint = [calculatePixelXFromCoordinateX(row[2]),calculatePixelYFromCoordinateY(row[1])]
            if max(abs(oldPoint[0]-newPoint[0]),abs(oldPoint[1]-newPoint[1])) > 1: #This removes the problem of bikes just sitting around doing nothing
                lineDrawerOnTheMapTop(oldPoint,newPoint,pixelList)             
                coordinates = [row[1],row[2]]
    editPictureColor(normalizePixelList(pixelList),picture,color)


    
#MAIN METHOD
def MapTheNLongestRuns(n,filename):# Controls the mapping of N longest runs (including saving/displaying the image)
    #The colours of different rides are different (or as different as the random number generator lets them be)
    
    seed(1)
    temporaryPixelList = [[0 for _ in range(width)] for _ in range(height)]
    editableImage = Image.open("MapOfTartuModified.png")
    LongestRunsOGDataframe = allTheRouteRows.sort_values(by=["length"],ascending=False)
    dictOfStopColors = {}
    
    for i in range(n):
        red = round(random() * 255)
        green = round(random() * 255)
        blue = round(random() * 255)
        
        start = LongestRunsOGDataframe.iat[i,5]
        end = LongestRunsOGDataframe.iat[i,6]
        
        if start not in dictOfStopColors:
            dictOfStopColors[start] = (red,green,blue)
        if end not in dictOfStopColors:
            dictOfStopColors[end] = (red,green,blue)
            
        LongestRouteDataframe = allTheLocRows.loc[allTheLocRows["route_code"] == LongestRunsOGDataframe.iat[i,0]]
        completeCoordinatesFieldsColor(LongestRouteDataframe,temporaryPixelList,editableImage,(red,green,blue))
    drawStationsOnMapColoured(editableImage,dictOfStopColors,filename)

In [18]:
#MAIN METHOD
#Method that does the entire process of creating and saving the map of top 12 bike stop origin to destinations pairs
#It needs a date in the format of "2019-08-06" and a filename with the ending ".png", for example "BigYoshi.png"
#It displays and saves the created map
def  findAndSaveTop12(filename,date):

    selectedRoute = allTheRouteRows.loc[(allTheRouteRows['lockedat'] == date) & (allTheRouteRows["length"] > 0.4)]
    selectedLoc = allTheLocRows.loc[(allTheLocRows['coord_date'] == date)]

    editableImage = Image.open("MapOfTartuModified.png")
    width, height = editableImage.size
    dictOfStopColors = {}
    colorsList = [(51,255,51),(51,51,255),(255,51,51),(255,153,51),(255,255,51),(153,255,51),(51,255,153),
                 (51,255,255),(51,153,255),(153,51,255),(255,51,255),(255,51,153)]
    colorsListDark = [(0,153,0),(0,0,153),(153,0,0),(153,76,0),(153,153,0),(76,153,0),(0,153,76),
                 (0,153,153),(0,76,153),(76,0,153),(153,0,153),(153,0,76)]
    temporaryPixelList = [[0 for _ in range(width)] for _ in range(height)]

    frequencydict = {}
    for i in range(selectedRoute.shape[0]):
        start = selectedRoute.iat[i,5]
        end = selectedRoute.iat[i,6]
        combination = start + "|" + end
        if combination not in frequencydict:
            frequencydict[combination] = 1
        else:
            frequencydict[combination] += 1

    sorted_freq = sorted(frequencydict.items(), key=operator.itemgetter(1))
    sorted_freq.reverse()

    for i in range(12):
        startAndEnd = sorted_freq[i][0]
        start = startAndEnd.split("|")[0]
        end = startAndEnd.split("|")[1]
        
        #By uncommenting the code below, one can more easily figure out where the real Metallica parking spot is
        #(Because it probably was not in middle of a lake/pond, like the coordinates say)
        
        #if start != "Metallica parkla" and end != "Metallica parkla":
        #    continue
        
        routeCodesIncluded = []
        for j in range(selectedRoute.shape[0]):
            startRow = selectedRoute.iat[j,5]
            endRow = selectedRoute.iat[j,6]
            if(startRow == start and endRow == end and selectedRoute.iat[i,7] > 0.4):
                routeCodesIncluded.append(selectedRoute.iat[j,0])
        bikeRoutesThatMatterDataframe = selectedLoc.loc[selectedLoc["route_code"].isin(routeCodesIncluded)]
        if start == end:
            completeCoordinatesFieldsColor(bikeRoutesThatMatterDataframe,temporaryPixelList,editableImage,colorsListDark[i])
            if start not in dictOfStopColors:
                dictOfStopColors[start] = colorsListDark[i]
            if end not in dictOfStopColors:
                dictOfStopColors[end] = colorsListDark[i]
        else:
            completeCoordinatesFieldsColor(bikeRoutesThatMatterDataframe,temporaryPixelList,editableImage,colorsList[i])
            if start not in dictOfStopColors:
                dictOfStopColors[start] = colorsList[i]
            if end not in dictOfStopColors:
                dictOfStopColors[end] = colorsList[i]

    drawStationsOnMapColoured(editableImage,dictOfStopColors,filename)

    
#MAIN METHOD
#Much like the method before, this one just doesn't need a date, but rather both a bike route and a bike location dataframe
#Ideally they would be correlated (from the same day/timeframe), as the method does not check that
def findAndSaveTop12GivenDataframe(filename,selectedRoute,selectedLoc): #For times where we want to limit the data to certain time or have to encompass 2+ days
    editableImage = Image.open("MapOfTartuModified.png")
    width, height = editableImage.size
    dictOfStopColors = {}
    colorsList = [(51,255,51),(51,51,255),(255,51,51),(255,153,51),(255,255,51),(153,255,51),(51,255,153),
                 (51,255,255),(51,153,255),(153,51,255),(255,51,255),(255,51,153)]
    colorsListDark = [(0,153,0),(0,0,153),(153,0,0),(153,76,0),(153,153,0),(76,153,0),(0,153,76),
                 (0,153,153),(0,76,153),(76,0,153),(153,0,153),(153,0,76)]
    temporaryPixelList = [[0 for _ in range(width)] for _ in range(height)]

    frequencydict = {}
    for i in range(selectedRoute.shape[0]):
        start = selectedRoute.iat[i,5]
        end = selectedRoute.iat[i,6]
        combination = start + "|" + end
        if combination not in frequencydict:
            frequencydict[combination] = 1
        else:
            frequencydict[combination] += 1

    sorted_freq = sorted(frequencydict.items(), key=operator.itemgetter(1))
    sorted_freq.reverse()

    for i in range(12):
        startAndEnd = sorted_freq[i][0]
        start = startAndEnd.split("|")[0]
        end = startAndEnd.split("|")[1]
        routeCodesIncluded = []
        for j in range(selectedRoute.shape[0]):
            startRow = selectedRoute.iat[j,5]
            endRow = selectedRoute.iat[j,6]
            if(startRow == start and endRow == end and selectedRoute.iat[i,7] > 0.4):
                routeCodesIncluded.append(selectedRoute.iat[j,0])
        bikeRoutesThatMatterDataframe = selectedLoc.loc[selectedLoc["route_code"].isin(routeCodesIncluded)]
        if start == end:
            completeCoordinatesFieldsColor(bikeRoutesThatMatterDataframe,temporaryPixelList,editableImage,colorsListDark[i])
            if start not in dictOfStopColors:
                dictOfStopColors[start] = colorsListDark[i]
            if end not in dictOfStopColors:
                dictOfStopColors[end] = colorsListDark[i]
        else:
            completeCoordinatesFieldsColor(bikeRoutesThatMatterDataframe,temporaryPixelList,editableImage,colorsList[i])
            if start not in dictOfStopColors:
                dictOfStopColors[start] = colorsList[i]
            if end not in dictOfStopColors:
                dictOfStopColors[end] = colorsList[i]

    drawStationsOnMapColoured(editableImage,dictOfStopColors,filename)


END OF METHODS:
START OF EXAMPLES ON HOW TO USE THE METHODS:
(DUE TO METHODS TAKING LIKE 5-10 MINUTES EACH TO COMPLETE, THEY ARE ORIGINALLY COMMENTED OUT)

In [19]:
#How to use the most common map making tool for one specific day, that takes a dataframe(in this case we give it a dataframe based on date)
    #and then both show and save different normalization types of that dataframe on a map.

#EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalization(createLocDataframeBasedOnDate("2019-07-18"),"MetallicaFinal")

In [20]:
#How to use the most common map making tool for specific time frames

#If the user values their time, they would not dare remove all 3 commented lines at the bottom at the same time, as it will take like 30 minutes

#MetallicaDay = createLocDataframeBasedOnDate("2019-07-18")
#before14 = MetallicaDay.loc[MetallicaDay["coord_time"] <= "14:00:00"]
#between14and223 = MetallicaDay.loc[(MetallicaDay["coord_time"] <= "22:30:00") & (MetallicaDay["coord_time"] > "14:00:00")]
#after223 = MetallicaDay.loc[MetallicaDay["coord_time"] > "22:30:00"]
#nextDay = createLocDataframeBasedOnDate("2019-07-19")
#nextDayMorning = nextDay.loc[nextDay["coord_time"] <= "04:00:00"]
#latenight = after223.append(nextDayMorning)


#EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalization(before14,"EarlyMorningMetallica")

#EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalization(between14and223,"MiddayMetallica")

#EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalization(latenight,"LatenightMetallica")

In [21]:
#How to use the most common map making tool to make an average map of many days instead of just one day

#averageDataframe1 = createLocDataframeBasedOnDate("2019-08-06")
#averageDataframe2 = createLocDataframeBasedOnDate("2019-07-25")
#averageDataframe3 = createLocDataframeBasedOnDate("2019-07-11")
#averageDataframe = averageDataframe1.append([averageDataframe2,averageDataframe3])


#EntireDrawingMapSavingProcessWithAndWithoutExponentialNormalizationAverage(averageDataframe,3,"FinalaverageDay")


In [22]:
#How to use the most common top12 origin/destination pair mapping method, that takes a date instead of two dataframes

#findAndSaveTop12("top12AverageDayRoadsFinal.png","2019-07-25")

In [23]:
#How to use the rarer top12 origin/destination pair mapping method, that takes two dataframes as parameters

#selectedRouteM = allTheRouteRows.loc[(allTheRouteRows['lockedat'] == "2019-07-18") & (allTheRouteRows["length"] > 0.4)
#                                        & (allTheRouteRows["lockedattime"] > "22:30:00")]
#selectedRouteM = selectedRouteM.append(allTheRouteRows.loc[(allTheRouteRows['lockedat'] == "2019-07-19") & (allTheRouteRows["length"] > 0.4)
#                                        & (allTheRouteRows["lockedattime"] < "04:00:00")])
#selectedLocM = allTheLocRows.loc[(allTheLocRows['coord_date'] == "2019-07-18")]
#selectedLocM = selectedLocM.append(allTheLocRows.loc[(allTheLocRows['coord_date'] == "2019-07-19")])

#findAndSaveTop12GivenDataframe("Top12AfterMetallicaRoutes.png",selectedRouteM,selectedLocM)

In [24]:
#How to use the method that maps N longest (distance wise) runs on the map.

#MapTheNLongestRuns(30,"BigChungus.png")